In [1]:
import os
import glob
import yaml #pip install pandas pyyaml
import pandas as pd
import mysql.connector #pip install mysql-connector-python
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Folder path                       
folder_path = r"D:\PROJECT_2\Data-Driven-Stock-Analysis\data"

# Month folders
month_name = [
    "2023-10","2023-11","2023-12","2024-01","2024-02","2024-03",
    "2024-04","2024-05","2024-06","2024-07","2024-08","2024-09","2024-10","2024-11"
]

# List to store all YAML data
records = []

# Loop through all month subfolders
for month in month_name:
    folder = os.path.join(folder_path, month)
    yaml_files = glob.glob(os.path.join(folder, "*.yaml"))

    for file_path in yaml_files:
        with open(file_path, "r") as file:
            data = yaml.safe_load(file)
            if data:
                # Append depending on type
                if isinstance(data, dict):
                    records.append(data)
                elif isinstance(data, list):
                    records.extend(data)
        print(f"Loaded {os.path.basename(file_path)} from {month}")


In [ ]:
# Convert all to one DataFrame
df1= pd.DataFrame(records)

print(f"\nTotal records loaded: {len(df1)}")
print(df1.head())


In [ ]:
df1.info()

In [20]:
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="0007"
    )

In [ ]:
# Establish connection
conn = get_connection()

# Create a cursor
cursor = conn.cursor()

# Create database
cursor.execute("CREATE DATABASE DATA")
print("Database 'DATA' created successfully!")

# Close connection
cursor.close()
conn.close()


In [ ]:
# Connect to MySQL
conn = get_connection()
cursor = conn.cursor()
cursor.execute("USE DATA")

# Create table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS MARKET (
        Ticker VARCHAR(50),
        close DECIMAL(10,2),
        date DATETIME,
        high DECIMAL(10,2),
        low DECIMAL(10,2),
        month VARCHAR(10),
        open DECIMAL(10,2),
        volume INT
    )
""")

# Commit and close
conn.commit()
cursor.close()
conn.close()

print("Table 'MARKET' created successfully!")

In [ ]:
conn = get_connection()
cursor = conn.cursor()
cursor.execute("USE DATA")
# Loop through DataFrame rows
for _, row in df1.iterrows():
    cursor.execute("""
        INSERT INTO MARKET (Ticker, close, date, high, low, month,open,volume)
        VALUES (%s, %s, %s, %s, %s,%s, %s,%s)
    """, (
        row.get("Ticker"),
        row.get("close"),
        row.get("date"),
        row.get("high"),
        row.get("low"),
        row.get("month"),
        row.get("open"),
        row.get("volume")
    ))

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully into 'MARKET'!")


In [ ]:
# Folder to save CSV files
output_folder =os.path.join(folder_path,"output_csv") #r"D:\data_driven\output_csv"
os.makedirs(output_folder, exist_ok=True)
conn = get_connection()
cursor = conn.cursor()
cursor.execute("USE DATA")

# Step 1: Get distinct tickers
cursor.execute("SELECT DISTINCT Ticker FROM MARKET")
companies = [row[0] for row in cursor.fetchall()]  # flatten to list
print("Tickers found:", companies)

# Step 2: Loop through each ticker, fetch data, and save as CSV
for ticker in companies:
    print(f"Processing {ticker}...")

    cursor.execute("""
        SELECT Ticker, close, date, high, low, month, open,volume
        FROM MARKET
        WHERE Ticker = %s
    """, (ticker,))
    rows = cursor.fetchall()

    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=["Ticker", "close", "date", "high", "low", "month", "open","volume"])

    # Step 3: Save to CSV
    csv_path = os.path.join(output_folder, f"{ticker}.csv")
    df.to_csv(csv_path, index=False)

    print(f"Saved: {csv_path}")

# Close connection
cursor.close()
conn.close()

print("✅ All ticker data exported to CSV successfully!")


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [27]:
output_folder = os.path.join(folder_path, "output_csv")
all_files = glob.glob(os.path.join(output_folder, "*.csv"))

list_val=[]
DATE_COL = 'date'
CLOSE_COL = 'close'

for filename in all_files:
    df2 = pd.read_csv(filename)
    ticker = os.path.splitext(os.path.basename(filename))[0].upper()
    df_cleaned = df2[[DATE_COL, CLOSE_COL]].copy()
    df_cleaned.rename(columns={DATE_COL: 'Date', CLOSE_COL: 'Close'}, inplace=True)
    df_cleaned['Ticker'] = ticker
    list_val.append(df_cleaned)

combined_df = pd.concat(list_val, axis=0, ignore_index=True)
combined_df.to_csv("combined_stock_data.csv", index=False)


In [ ]:
combined_df.info()

In [29]:
v=combined_df['Ticker'].unique()
len(v)

50

In [30]:

combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df['Close'] = pd.to_numeric(combined_df['Close'], errors='coerce')
combined_df = combined_df.dropna(subset=['Close'])
combined_df = combined_df.sort_values(by=['Ticker', 'Date'])

combined_df['Daily_Return'] = combined_df.groupby('Ticker')['Close'].pct_change()
returns_df = combined_df.dropna(subset=['Daily_Return'])
volatility = returns_df.groupby('Ticker')['Daily_Return'].std().reset_index()
volatility.columns = ['Ticker', 'Volatility']
top_10_volatility = volatility.sort_values(by='Volatility', ascending=False).head(10)

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(top_10_volatility['Ticker'], top_10_volatility['Volatility'], color='teal')
plt.xlabel('Stock Ticker')
plt.ylabel('Volatility (Standard Deviation of Daily Returns)')
plt.title('Top 10 Most Volatile Stocks')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('top_10_volatility_bar_chart.png')


In [ ]:


# --- READ COMBINED CSV FILES ---
output_folder = os.path.join(folder_path, "output_csv")
all_files = glob.glob(os.path.join(output_folder, "*.csv"))

list_val = []
DATE_COL = 'date'
CLOSE_COL = 'close'

for filename in all_files:
    df2 = pd.read_csv(filename)
    ticker = os.path.splitext(os.path.basename(filename))[0].upper()

    df_cum = df2[[DATE_COL, CLOSE_COL]].copy()
    df_cum.rename(columns={DATE_COL: 'Date', CLOSE_COL: 'Close'}, inplace=True)
    df_cum['Ticker'] = ticker

    list_val.append(df_cum)

cum_df = pd.concat(list_val, axis=0, ignore_index=True)

# Sort by date (important!)
cum_df['Date'] = pd.to_datetime(cum_df['Date'])
cum_df.sort_values(by=['Ticker', 'Date'], inplace=True)

# Calculate cumulative return
cum_df['Start_Close'] = cum_df.groupby("Ticker")['Close'].transform('first')
cum_df['Cumulative_Return'] = (cum_df['Close'] / cum_df['Start_Close']) - 1
cum_df['Cumulative_Return_Percent'] = cum_df['Cumulative_Return'] * 100

# ------------- SELECT TOP 5 STOCKS -----------------

# Get last cumulative return value per stock
final_cum = cum_df.groupby("Ticker")['Cumulative_Return'].last().reset_index()

# Pick top 5 performing stocks
top5_tickers = final_cum.sort_values(by='Cumulative_Return', ascending=False).head(5)['Ticker']
print("Top 5 Stocks Based on Cumulative Return:")
print(top5_tickers)

# Filter data for only these 5 stocks
top5_df = cum_df[cum_df['Ticker'].isin(top5_tickers)]

# ------------- PLOT CUMULATIVE RETURNS --------------

plt.figure(figsize=(14, 7))

for ticker in top5_tickers:
    df_plot = top5_df[top5_df['Ticker'] == ticker]
    plt.plot(df_plot['Date'], df_plot['Cumulative_Return'], label=ticker)

plt.title("Cumulative Return Over Time - Top 5 Performing Stocks")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# ----------------------------
# 1. Load sector mapping
# ----------------------------
sector_map = pd.read_csv(r"D:\data_driven\.dds\Sector_data - Sheet1.csv")  # stock, sector

# ----------------------------
# 2. Load combined stock price dataset
# ----------------------------
output_folder = os.path.join(folder_path, "output_csv")
all_files = glob.glob(os.path.join(output_folder, "*.csv"))

sect_tick=[]
all_daa=[]
for filename in all_files:
    df2 = pd.read_csv(filename)
    ticker = os.path.splitext(os.path.basename(filename))[0].upper()
    df_cleaned = df2.copy()
    all_daa.append(df_cleaned)
    sect_tick.append(ticker)

combined_df = pd.concat(all_daa, axis=0, ignore_index=True)
combined_df.to_csv("sector_ticker.csv", index=False)  # date, stock, close
combined_df['date'] = pd.to_datetime(combined_df['date'])

n=pd.DataFrame(sect_tick,columns=["Ticker"])
sector_map["COMPANY"]=n["Ticker"]
sector_map.rename(columns={"COMPANY":"Ticker"},inplace=True)
# ----------------------------
# 3. Calculate yearly return for each stock
# ----------------------------
# Get first and last price for each stock
first_last = combined_df.sort_values('date').groupby('Ticker').agg(
    first_price=('close', 'first'),
    last_price=('close', 'last')
)

first_last['yearly_return'] = (first_last['last_price'] - first_last['first_price']) / first_last['first_price'] * 100
first_last.reset_index(inplace=True)

# ----------------------------
# 4. Merge sector data
# ----------------------------
merged = pd.merge(first_last, sector_map, on='Ticker', how='left')

# ----------------------------
# 5. Calculate average yearly return by sector
# ----------------------------
sector_perf = merged.groupby('sector')['yearly_return'].mean().sort_values(ascending=False)

print("\nAverage Sector-wise Returns:")
print(sector_perf)

# ----------------------------
# 6. Plot bar chart
# ----------------------------
plt.figure(figsize=(12,6))
sector_perf.plot(kind='bar')

plt.title("Average Yearly Return by Sector")
plt.xlabel("Sector")
plt.ylabel("Average Yearly Return (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [33]:
folder_path_1=r"D:\data_driven\data\Sector_data - Sheet1.csv"
def avg_year_returns(folder_path,folder_path_1):
    sector_map = pd.read_csv(folder_path_1)  # stock, sector
    output_folder = os.path.join(folder_path, "output_csv")
    all_files = glob.glob(os.path.join(output_folder, "*.csv"))
    sect_tick=[]
    all_daa=[]
    for filename in all_files:
        df2 = pd.read_csv(filename)
        ticker = os.path.splitext(os.path.basename(filename))[0].upper()
        df_cleaned = df2.copy()
        all_daa.append(df_cleaned)
        sect_tick.append(ticker)
    combined_df = pd.concat(all_daa, axis=0, ignore_index=True)
    combined_df.to_csv("sector_ticker.csv", index=False)  # date, stock, clos
    combined_df['date'] = pd.to_datetime(combined_df['date'])
    n=pd.DataFrame(sect_tick,columns=["Ticker"])
    sector_map["COMPANY"]=n["Ticker"]
    sector_map.rename(columns={"COMPANY":"Ticker"},inplace=True)

    first_last = combined_df.sort_values('date').groupby('Ticker').agg(
        first_price=('close', 'first'),
        last_price=('close', 'last'))
    first_last['yearly_return'] = (first_last['last_price'] - first_last['first_price']) / first_last['first_price'] * 100
    first_last.reset_index(inplace=True)
    merged = pd.merge(first_last, sector_map, on='Ticker', how='left')
    sector_perf = merged.groupby('sector')['yearly_return'].mean().sort_values(ascending=False)
    print("\nAverage Sector-wise Returns:")
    print(sector_perf)
    plt.figure(figsize=(12,6))
    sector_perf.plot(kind='bar')
    plt.title("Average Yearly Return by Sector")
    plt.xlabel("Sector")
    plt.ylabel("Average Yearly Return (%)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()



In [34]:
# --- READ ALL STOCK CSV FILES ---
def stock_correlation(folder_path):
    output_folder = os.path.join(folder_path, "output_csv")
    all_files = glob.glob(os.path.join(output_folder, "*.csv"))

    stock_dict = {}   # Ticker → its closing prices

    for filename in all_files:
        df = pd.read_csv(filename)
        
        # Extract ticker name from filename
        ticker = os.path.splitext(os.path.basename(filename))[0].upper()

        # Clean dataframe: convert date & sort
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date')

        # Set date as index
        df = df.set_index('date')

        # Keep only the Close column
        stock_dict[ticker] = df['close']

    # Combine all stocks into a single DataFrame by date
    combined_df = pd.DataFrame(stock_dict)

    # Compute correlation matrix
    corr_matrix = combined_df.corr()

    print("\n🔍 STOCK PRICE CORRELATION MATRIX:")
    print(corr_matrix)

    # ---------------- PLOT HEATMAP -------------------
    plt.figure(figsize=(12, 8))
    plt.imshow(corr_matrix, cmap="coolwarm", interpolation="nearest")
    plt.colorbar(label="Correlation Coefficient")

    # Add ticks and labels
    plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
    plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)

    plt.title("Stock Price Correlation Heatmap")
    plt.tight_layout()
    plt.show()

    return corr_matrix


In [35]:
output_folder = os.path.join(folder_path, "output_csv")
all_files = glob.glob(os.path.join(output_folder, "*.csv"))


In [36]:
all_daa=[]
for filename in all_files:
    df2 = pd.read_csv(filename)
    df_cleaned = df2.copy()
    all_daa.append(df_cleaned)
combined_df = pd.concat(all_daa, axis=0, ignore_index=True)

In [ ]:
combined_df.groupby(["Ticker","month"])["close"].agg("mean")

In [39]:
all_data = []

    # Read all CSV files and combine
for file in all_files:
    ticker = os.path.splitext(os.path.basename(file))[0].upper()
    df = pd.read_csv(file)

    df['date'] = pd.to_datetime(df['date'])
    df['Ticker'] = ticker
    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)


In [ ]:
monthly = combined_df.sort_values('date').groupby(['Ticker', 'Month'])['close'].agg(['first', 'last'])
monthly

In [41]:

    # ---- Extract month in YYYY-MM format ----
combined_df['Month'] = combined_df['date'].dt.to_period('M').astype(str)

    # ---- Calculate Monthly Returns ----
monthly = combined_df.sort_values('date').groupby(['Ticker', 'Month'])['close'].agg(['first', 'last'])

monthly['Monthly_Return'] = ((monthly['last'] - monthly['first']) / monthly['first']) * 100
monthly.reset_index(inplace=True)

In [ ]:
unique_months = monthly['Month'].unique()

    # Create 12 charts (one per month)
for month in unique_months:
        df_month = monthly[monthly['Month'] == month]

        top5_gainers = df_month.nlargest(5, 'Monthly_Return')
        top5_losers = df_month.nsmallest(5, 'Monthly_Return')

        # ---- Plot ----
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))

        # Gainers
        axes[0].barh(top5_gainers['Ticker'], top5_gainers['Monthly_Return'])
        axes[0].set_title(f"Top 5 Gainers - {month}")
        axes[0].set_xlabel("Monthly Return (%)")

        # Losers
        axes[1].barh(top5_losers['Ticker'], top5_losers['Monthly_Return'], color='red')
        axes[1].set_title(f"Top 5 Losers - {month}")
        axes[1].set_xlabel("Monthly Return (%)")

        plt.tight_layout()
        plt.show()
